In [1]:
import pandas as pd
import os

# Load the Excel file
file_path = "Reviews sample set.xlsx"
review_data = pd.read_excel(file_path, sheet_name="Sheet1")

In [2]:
review_data

,review_time,reviewer_name,rating,review_text
0,2024-12-30T11:11:49.654205Z,RAVI REVATHI,FIVE,All the non-veg dishes here are amazing. Food ...
1,2024-12-27T16:21:40.323829Z,Gaurav Choudhury,FIVE,Awesome food and amazing people. Being a north...
2,2024-12-27T11:02:37.626451Z,Rio Vicky,FIVE,Food was too good and services also well🫶🏻My p...
3,2024-12-27T10:42:53.286731Z,Ravi prasath S,FIVE,Food is ready Good and feels like home food
4,2024-12-26T18:40:28.677557Z,Lakshminarayana Raju,FIVE,NaN
...,...,...,...,...
371,2025-08-23T11:54:59.294189Z,Hemu Hema,FIVE,Chukka briyani was very good and sambar rice w...
372,2025-08-23T11:52:46.891593Z,Nagarajan Nagarajan,FIVE,Justin good person and service
373,2025-08-24T05:57:49.942489Z,dj kannan,FIVE,Awesome enjoyed the dinner prawn chukka briya...
374,2025-08-24T10:33:23.505059Z,Subbu Raj,FIVE,Good


In [3]:
positive_review = review_data[review_data['rating'].str.upper()=="FIVE"].copy()
positive_review

,review_time,reviewer_name,rating,review_text
0,2024-12-30T11:11:49.654205Z,RAVI REVATHI,FIVE,All the non-veg dishes here are amazing. Food ...
1,2024-12-27T16:21:40.323829Z,Gaurav Choudhury,FIVE,Awesome food and amazing people. Being a north...
2,2024-12-27T11:02:37.626451Z,Rio Vicky,FIVE,Food was too good and services also well🫶🏻My p...
3,2024-12-27T10:42:53.286731Z,Ravi prasath S,FIVE,Food is ready Good and feels like home food
4,2024-12-26T18:40:28.677557Z,Lakshminarayana Raju,FIVE,NaN
...,...,...,...,...
371,2025-08-23T11:54:59.294189Z,Hemu Hema,FIVE,Chukka briyani was very good and sambar rice w...
372,2025-08-23T11:52:46.891593Z,Nagarajan Nagarajan,FIVE,Justin good person and service
373,2025-08-24T05:57:49.942489Z,dj kannan,FIVE,Awesome enjoyed the dinner prawn chukka briya...
374,2025-08-24T10:33:23.505059Z,Subbu Raj,FIVE,Good


In [4]:
from openai import OpenAI

# Load API key safely from environment variable
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def generate_reply(review_text, reviewer_name):
    """Generate a short reply for positive reviews only."""
    prompt = f"""
    The user review is: "{review_text}"

    Write a short, human-like reply in max 2 lines.
    - Thank them.
    - Mention something from the review (like food, service, etc.).
    - Keep it warm and genuine.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=800,
        temperature=0.7,
    )

    return response.choices[0].message.content.strip()

In [5]:
# # Generate replies
# positive_reviews["reply"] = positive_reviews.apply(
#     lambda row: generate_reply(str(row["review_text"]), row["reviewer_name"]), axis=1
# )

# # Save results to a new Excel file
# output_path = "Replies_to_Positive_Reviews.xlsx"
# positive_reviews.to_excel(output_path, index=False)

# print(f"✅ Replies generated and saved at: {output_path}")


###  I found error after running the open ai API models 

### RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### So i used offilne api model 

In [6]:
import ollama
import time

def generate_reply(review_text, reviewer_name):
    """Generate a reply using LLaMA-3 via Ollama."""
    prompt = f"""
    The user review is: "{review_text}"
    
    Write a short, warm, and genuine reply as if you are the restaurant owner. 
    Address the reviewer by their name: {reviewer_name}.
    Do not use placeholders like [Your Name] or [Your Business Name].
    Just write the final reply message directly (2-3 friendly sentences).
    """
    response = ollama.chat(
        model="llama3:8b",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['message']['content'].strip()

In [7]:
# --- Process in batches (avoid overloading GPU) ---
batch_size = 10
all_replies = []

for i in range(0, len(positive_review), batch_size):
    batch = positive_review.iloc[i:i+batch_size].copy()
    
    print(f" Processing batch {i//batch_size + 1} ({len(batch)} reviews)...")

    # EMPTY LIST TO STORE current batch reply
    replies = []
    
    for index, row in batch.iterrows():
        review_text = str(row["review_text"])
        reviewer_name = row["reviewer_name"]
        reply = generate_reply(review_text,reviewer_name)
        replies.append(reply)
    batch["reply"] = replies
    
    all_replies.append(batch)
    
    time.sleep(2)  # short pause, keeps GPU stable

# --- Merge results ---
reviews_with_reply = pd.concat(all_replies)
reviews_with_reply.to_excel("reviews_with_replies.xlsx", index=False)

print("Replies generated and saved to reviews_with_replies.xlsx")

 Processing batch 1 (10 reviews)...
 Processing batch 2 (10 reviews)...
 Processing batch 3 (10 reviews)...
 Processing batch 4 (10 reviews)...
 Processing batch 5 (10 reviews)...
 Processing batch 6 (10 reviews)...
 Processing batch 7 (10 reviews)...
 Processing batch 8 (10 reviews)...
 Processing batch 9 (10 reviews)...
 Processing batch 10 (10 reviews)...
 Processing batch 11 (10 reviews)...
 Processing batch 12 (10 reviews)...
 Processing batch 13 (10 reviews)...
 Processing batch 14 (10 reviews)...
 Processing batch 15 (10 reviews)...
 Processing batch 16 (10 reviews)...
 Processing batch 17 (10 reviews)...
 Processing batch 18 (10 reviews)...
 Processing batch 19 (10 reviews)...
 Processing batch 20 (10 reviews)...
 Processing batch 21 (10 reviews)...
 Processing batch 22 (10 reviews)...
 Processing batch 23 (10 reviews)...
 Processing batch 24 (10 reviews)...
 Processing batch 25 (10 reviews)...
 Processing batch 26 (10 reviews)...
 Processing batch 27 (10 reviews)...
 Processin